Agents can be unreliable and may need human input to successfully accomplish tasks. Similarly, for some actions, you may want to require human approval before running to ensure that everything is running as intended.

LangGraph supports `human-in-the-loop` workflows in a number of ways. In this section, we will use LangGraph's `interrupt_before` functionality to always break the tool node.

First, start from our existing code. The following is copied from Part 3.

In [1]:
import os
from getpass import getpass

from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode, tools_condition

from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_groq import ChatGroq

In [2]:
os.environ["LANGCHAIN_PROJECT"] = "04. Human in the loop."

def _set_env(var: str):
  if not os.environ.get(var):
    os.environ[var] = getpass(f"Enter {var}: ")

_set_env("TAVILY_API_KEY")
_set_env("GROQ_API_KEY")

Enter TAVILY_API_KEY:  ········


In [12]:
memory = SqliteSaver.from_conn_string(":memory:")

class State(TypedDict):
    messages: Annotated[list, add_messages]

#Create graph builder.

graph_builder = StateGraph(State)

tool = TavilySearchResults(max_results=3)
tools = [tool]

llm = ChatGroq(temperature = 0.1, model = "Llama3-70b-8192")
llm_with_tools = llm.bind_tools(tools)

In [13]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools = [tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

In [14]:
#Compile the graph specifying to interrupt_before the action node.
graph = graph_builder.compile(
    checkpointer = memory,
    interrupt_before = ["tools"])

In [15]:
user_input = "I'm learning LangGraph. Could you do some research on it for me?"

config = {"configurable": {"thread_id": "1"}}

#The config is the 2nd positional argument to stream() or invoke()
events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode = "values")


for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_bwpx)
 Call ID: call_bwpx
  Args:
    query: LangGraph


***
Let's inspect the graph state to confirm it worked.

In [16]:
snapshot = graph.get_state(config)
snapshot.next

('tools',)

In [17]:
existing_message = snapshot.values["messages"][-1]
existing_message.tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'LangGraph'},
  'id': 'call_bwpx',
  'type': 'tool_call'}]


This query seems reasonable. Nothing to filter here. The simplest thing the human can do is just let the graph continue executing. Let's do that below.

Next, continue the graph! Passing in `None` will just let the graph continue where it left off, without adding anything new to the state.

In [18]:
# `None` will append nothing new to the current state, letting it resume as if it had never been interrupted

events = graph.stream(None, config, stream_mode = "values")

for event in events:
    if "messages" in events:
        event["messages"][-1].pretty_print()